<a href="https://colab.research.google.com/github/JanEggers-hr/ddj-python-kurs/blob/main/gude_welt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# gude_welt.ipynb
Ein erstes produktives Python-Skript.

Für das Seminar "Datenjournalismus" im Sommersemester 2020
Hochschule Darmstadt, Studiengang Onlinejournalismus, 6. Semester

CC-BY Jan Eggers

## Was man über "Notebooks" wissen muss
- Notebooks sind eine Mischung aus ausführbarem Code und Textblöcken.
- Den Text gibt man als ["Markdown"](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) ein - mit einfachen Steuerbefehlen wie **\*\*fett\*\*** oder **\# Überschrift**. 
- Wenn man einen Codeblock ausführt, gibt er das Ergebnis unter dem Block aus - wie es Python auf der Kommandozeile auch tun würde.
- Man kann die Codeblöcke mit Shift+Enter ausführen und zum nächsten springen.

Jetzt einfach mal ausprobieren: 
*Kursiv* und **fett** 


In [ ]:
1+1

Und die traditionelle Begrüßung in einer neuen Programmiersprache: 

In [ ]:
print("Gude Welt!")

## Vorbereitung: Bibliotheken laden
Zusatzpakete statten Python mit Fähigkeiten zur Ein- und Ausgabe aus. Sie müssen in der jeweiligen Python-Umgebung installiert sein - wenn sie das nicht sind, muss man einmal auf die Kommandozeile und beispielsweise mit dem Befehl
```conda install pandas```
das Pandas-Paket installieren, das wir für Tabellen ("Dataframes") brauchen. 

(Wer Updates für ein Paket einspielen will, nutzt dazu den Befehl ```conda update ...``` oder einfach  ```conda update --all```)

In [ ]:
import pandas as pd
import numpy as np

Den obigen Textblock ausgeführt - und nichts ist passiert? Dann hat alles geklappt - und wir können loslegen. 

## Teil 1: CSV-Datei einlesen
Wir lesen die CSV-Datei `plz-hessen.csv` in eine neue Variable namens plz_df und schauen sie uns danach kurz an.

`plz-hessen.csv` ist ein gutartiges CSV: 
- Als Trennzeichen wird das Semikolon (";") verwendet, das keine Probleme mit Kommazahlen bereitet
- Text ist in Anführungszeichen - und er enthält auch keine Steuerzeichen für Zeilensprünge, die gerne für Chaos sorgen
- Der Zeichensatz der Datei ist UTF-8 - im Universal-Format gibt es keine Probleme mit Umlauten etc. 

Dass unser CSV nicht "comma-separated" ist, sondern "Semikolon-separated", müssen wir dem Befehl über den Zusatz `delimiter=";"` mitgeben.

In [ ]:
plz_df = pd.read_csv("plz-hessen.csv",delimiter=";")
plz_df

Prima! Wir sehen, dass die Tabelle nur zwei Spalten enthält: Eine Spalte "PLZ" mit den Postleitzahlen, und eine Spalte "Ort" mit den Ortsnamen.

Schauen wir uns mal die Dateitypen an. Dazu nutzen wir "Eigenschaften" und "Methoden" - Befehle, die wir direkt an den Variablennamen anhängen, durch einen Punkt getrennt, und die dem Computer sagen: Mach etwas mit dieser Variable. 

Was man mit einer Variable machen kann, hängt vom Dateityp ab - hier, bei unserer "Dataframe"- (Tabellen-) Variable `plz_df`, nutzen wir
- eine Methode, die uns die Anzahl der Spalten zurückgibt - `plz_df.columns`
- eine Methode, die den Dateityp einer Dataframe-Spalte zurückgibt - `plz_df.dtype`

In [ ]:
print("Der Dateityp von plz_df ist",type(plz_df))

print("Die Dateitypen der Spalten des Dataframes: ")
for y in plz_df.columns:
    print(y,plz_df[y].dtype)

Dann können wir versuchen, die zweite Tabelle einzulesen, `ags-plz-lat-lon.csv`. 
Ich sag's gleich, das wird etwas schwieriger:
- Sie nutzt Kommas.
- Sie hat keine Anführungszeichen. 
- Sie nutzt einen anderen Zeichensatz. 

Aufgabe: Den nächsten Befehl ausprobieren - und so korrigieren, dass er funktioniert. 

In [ ]:
gemeinden_df = pd.read_csv("ags-plz-lat-lon.csv",delimiter=",")
gemeinden_df

Okay, so geht's nicht - also den `delimiter` auf Komma ändern, das Komma auf  und in der [Pandas-Dokumentation für read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) nach dem richtigen Befehlszusatz suchen. (Das richtige Encoding für diese Datei ist übrigens der Windows-Zeichensatz `ISO-8859-1`.)

Also versuchen wir's nochmal: 

In [ ]:
gemeinden_df = pd.read_csv("ags-plz-lat-lon.csv",delimiter=",",encoding="ISO-8859-1",decimal=",")
gemeinden_df

Jetzt werfen wir alle Zeilen, in denen Werte fehlen (was man an `NaN` sieht, "Not a Number") aus dem Dataframe. Wie wir das machen, [ergooglen wir uns einfach](https://lmgtfy.com/?q=python+drop+na+rows).

In [ ]:
gemeinden_df = gemeinden_df.dropna()
gemeinden_df

**Das hat funktioniert!**

Allerdings wollen wir ja nur Hessen - also ziehen wir mal nur raus, was im Bundesland 6 (Hessen) spielt: 

In [ ]:
gemeinden_df = gemeinden_df.query("Land == 6")
gemeinden_df

Auch das hat funktioniert.

Allerdings: Wir haben noch keine AGS - wir müssen sie uns aus den Zahlenwerten für Land, Kreis und Gemeinde zusammensetzen, jeweils als String. (Es wäre einfacher gewesen, wir hätten die String-Wert in der Tabelle behalten!)

In [ ]:
ags_df = gemeinden_df.copy()
for i in ags_df.index:
    ags_df.loc[i,'AGS'] = "0"+str(int(ags_df.Land[i]*1000000+ags_df.RB[i]*100000+ags_df.Kreis[i]*1000+ags_df.Gem[i]))
ags_df

*Eine kleine Anmerkung für alle, die schon programmiert haben und Python bisher nicht kennen: Vermutlich habt ihr euch über das ```.loc``` hinter dem Variablennamen gewundert - warum haben wir die Zelle im Dataframe nicht einfach über ```ags_df\[spaltenname\]\[indexnr.\] = wert``` verändert?*

*Die Antwort hängt damit zusammen, dass Python die beiden Teile eines solchen Adressierungs-Befehls unabhängig voneinander ausführt - aus Sicht des Computers hieße das: Mach mal eine Kopie von einem Teil des Dataframes, und dann weise wieder da einem Teil davon etwas zu. So ist aber nicht sichergestellt, dass wir wirklich auf unsere Originaldaten schreiben - diese Verkettung von Befehlen finde ich wahnsinnig irritierend, ist aber halt so. Deshalb nutzen wir die Methode ```.loc```, die die Adressierung in einem Schritt erledigt. Was es mir ganz gut erklärt hat, war [dieser Artikel](https://www.dataquest.io/blog/settingwithcopywarning/).*

In [ ]:
# Das hier funktionert nicht bzw. produziert eine beeindruckende Warnung: 
for i in ags_df.index:
    ags_df['AGS'] = "0"+str(int(ags_df.Land[i]*1000000+ags_df.RB[i]*100000+ags_df.Kreis[i]*1000+ags_df.Gem[i]))

Eins müssen wir noch reparieren: die Spalte `Bevoelkerung` enthält keine Zahl, sondern eine Zeichenkette, weil das Statistikamt gerne Leerzeichen als Trennzeichen benutzt. 

(Und wenn wir oben beim Import nicht noch `decimal=","` angegeben hätten, um Python mitzuteilen, dass im Deutschen ein Komma als Dezimalpunkt dient, dann wären `Flaeche`, `Lat` und `Lon` auch noch Zeichenketten, weil sonst das Komma in den Koordinaten für Konfusion gesorgt hätte - seufz. 

Beweis gefällig? 

In [ ]:
print("Die Dateitypen der Spalten des Dataframes: ")
ags_df.dtypes

Bevoelkerung ist ein ```object```, keine Zahl. Also müssen wir die Leerzeichen aus der Spalte `Bevoelkerung` werfen und dann alles in den richtigen Datentyp - int64 - umwandeln: 

In [ ]:
for i in ags_df.index:
    temp = ags_df.loc[i,'Bevoelkerung']    # String rausziehen
    temp = str.replace(temp," ","")        # Leerzeichen entfernen
    ags_df.loc[i,'Bevoelkerung']=pd.to_numeric(temp)
    
ags_df
#= ags_df.Bevoelkerung.str.replace(" ","")
# ags_df.Bevoelkerung = pd.to_numeric(ags_df.Bevoelkerung)

Hat geklappt!

Jetzt noch ein bisschen aufräumen: die Postleitzahl in einen Integer-Wert umwandeln (eine ganze Zahl statt der ```float```-Kommazahl, die da steht), und die relevanten Spalten aussuchen. 

Dazu definieren wir zunächst eine Listen-Variable mit allen Spaltennamen, die wir haben wollen - und suchen dann aus, mit der Methode ```filter()```.

(Der Zwischenschritt mit der Variable ```spalten``` ist nicht nur dazu da, um den Variablen-Typ "Liste" vorzustellen, sondern auch, weil ```filter()``` nicht mehr als fünf Parameter verarbeiten kann.)

In [ ]:
ags_df['PLZ'] = ags_df['PLZ'].astype(int)
spalten = ['AGS','Name','Flaeche','Bevoelkerung','PLZ','Lon','Lat','Besiedelung']
ags_df = ags_df.filter(spalten)
ags_df

**Fertig!**  

Jetzt führen wir die beiden Tabellen zusammen. 

## Teil 2: Tabellen zusammenführen

Nehmen wir mal an, wir wollen eine Tabelle, in der man über die Postleitzahl die Stadt raussuchen kann - mit Bevölkerungszahl und Geokoordinaten. 

Wir erinnern uns, dass die Namen in der offiziellen Gemeindestatistik manchmal ein wenig wild sind. Wir brauchen also einen Ortsnamen, mit dem wir arbeiten können - den, der auch in der PLZ-Datei steht. Aber wie finden wir den passenden raus?

Wer genau hingeschaut hat, hat gesehen: Auch die Tabelle `ags_df` enthält eine Postleitzahl (aber nur eine). Um wirklich für jede Postleitzahl eine Zeile in der Tabelle haben, müssen wir zwei Schritte gehen: 

- Erst mal den "Ort" aus der PLZ-Tabelle ```plz_df``` in die Tabelle mit aufnehmen. 
- Dann Zeilen für alle PLZ-Werte für den entsprechenden Ort anlegen.

In [ ]:
ags_mit_ort_df = pd.merge(left=ags_df, right=plz_df, left_on='PLZ', right_on='PLZ', how='left')
ags_mit_ort_df

Die Spalte "Ort" können wir jetzt verwenden, um nochmal einen ```merge()``` mit der Postleitzahl-Tabelle zu machen - nur diesmal als "right merge", wo alle Werte aus der "rechten" Tabelle (also der PLZ-Tabelle) verwendet werden sollen.

In [ ]:
ags_mit_ort_df[ags_mit_ort_df.Name.str.find("Bad") != -1]
#ags_mit_ort_df[ags_mit_ort_df.AGS=="06434001"]

In [ ]:
zusammen_df = pd.merge(left=ags_mit_ort_df, right=plz_df, left_on='Ort', right_on='Ort', how='right')
zusammen_df

## Teil 3: Tabelle als Excel-Datei ausgeben
Und jetzt hätten wir das Ganze gerne wieder als Excel-Datei. Das ist zum Glück einfach.

Die Parameter sagen: Spaltennamen mit in die Tabelle schreiben, Zeilennummern (den Index) nicht - bei anderen Pandas-Tabellen kann der Index wichtige Daten enthalten, deshalb ist das wichtig. Und Zahlen mit Komma ausgeben! Alle Formatierungsbefehle für `to_excel` finden sich [in der Pandas-Dokumentation.](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_excel.html) 

In [ ]:
zusammen_df.to_excel ('export.xlsx', index = False, header=True)

## Teil 4: Zusatzaufgabe

Bis hierher gekommen? Herzlichen Glückwunsch! Dann mal diesen Code hier ausführen: 

In [ ]:
cyphertext = b'R3V0ZW4gTW9yZ2VuISBJY2ggaG9mZmUsIGFsbGUgc2luZCBnZXN0ZXJuIGVpbmlnZXJtYcOfZW4ga2xhciBnZWtvbW1lbiAtIHVuZCBpaHIga29ubnRldCBldWNoIGdlZ2Vuc2VpdGlnIGhlbGZlbi4gRGFua2UgZsO8ciBkaWUgQmVyZWl0c2NoYWZ0LCBzaWNoIGRhcmF1ZiBlaW56dWxhc3NlbiEgLS0gRGllIEJlcmVjaHRpZ3VuZ2VuIGbDvHIgTGltZVN1cnZleSBoYWJlIGljaCBoZXV0ZSBtb3JnZW4gbm9jaCBtYWwgYW5nZXBhc3N0OyBqZXR6dCBtw7xzc3RlIGVzIHp1bWluZGVzdCBiZWkgZGVuIG1laXN0ZW4gZnVua3Rpb25pZXJlbi4gQml0dGUgbWVsZGVuLCB3ZW5uIG5pY2h0LiBVbmQgZW50c2NodWxkaWd0LCBkYXNzIGljaCBkYXMgZ2VzdGVybiBuaWNodCBtZWhyIGdlc2NoYWZmdCBoYWJlOyBkYW5rZSBmw7xyIGV1cmUgR2VkdWxkLg=='

import base64
print(base64.standard_b64decode(cyphertext).decode())

**Nur Mut! Man kann nichts kaputt machen!**